## Not really an intro from me...

I choose house prices as my first kernel on Kaggle because I hope to go over the entire analytic process in this kernel. We all know that data cleaning is the dirtist part of a whole project but it is also the foundation for further study. So you can see that the data cleaning part occupies most of the space. There are far more techniques that can be incorporated and more precise analyses can be done. I just point out the basic steps I think are mostly important. Now let us start!

Roadmap:
1. Import data
1. Preliminary analysis
    * For numerical variables: descriptive stats, distribution and normality test, correlation analysis
    * For categorical variables: frequency table, correlation analysis 
1. Data preparation
    * Missing data imputation
    * For numerical variables: standardization or Yeo-Johnson transformation
    * For categorical varaibles: one-hot encoding
    * Principal component analysis
1. Modeling
    * Random Forests
    * XGBoost
    * Neural Networks

## Import Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('notebook', font_scale=1.5)
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

train.head()

In [ ]:
train.shape

In [ ]:
test.head()

In [ ]:
test.shape

## Preliminary Analysis

### Numeric Variables

I split the analysis based on the different types of variables. It is easier to describe the numeric variables since we can calculate descriptive statistics and apply various statistical tests.

In [ ]:
train_num = train.select_dtypes(exclude=['object']).columns
train_num

#### 1. Basic statistics

In [ ]:
train.describe()

The first line "count" indicates several variables that have missing values given the total number of observations is 1460, - LotFrontage, MasVnrArea, GarageYrBlt. LotFrontage's missing values are the most, which is 259. Judged from mean, quantiles, etc., we can also identify many skewed variables, for example, BsmtFinSF1 (Type 1 finished square feet), BsmtFullBath (Basement full bathrooms), LotArea (Lot size in square feet), MasVnrArea (Masonry veneer area in square feet).

#### 2. Check the distribution

In [ ]:
from scipy.stats import shapiro
stat, p = shapiro(train['BsmtFinSF1'])
print('Skewness=%.3f' %train['BsmtFinSF1'].skew())
print('Statistics=%.3f, p=%.3f' %(stat, p))

alpha = 0.05
if p > alpha:
    print('Data looks normal (fail to reject H0)')
else:
    print('Data does not look normal (reject H0)')

sns.distplot(train['BsmtFinSF1']);

**BsmtFinSF1** is highly skewed and its distribution is not normal based on the Shapiro-Wilk test.

In [ ]:
stat, p = shapiro(train['BsmtFullBath'])
print('Skewness=%.3f' %train['BsmtFullBath'].skew())
print('Statistics=%.3f, p=%.3f' %(stat, p))

sns.distplot(train['BsmtFullBath']);

**BsmtFullBath** is moderately skewed but its distribution is not normal either because p value is smaller than 0.05. Acutally we can observe its distribution in the plot which has two peaks.

In [ ]:
stat, p = shapiro(train['LotArea'])
print('Skewness=%.3f' %train['LotArea'].skew())
print('Statistics=%.3f, p=%.3f' %(stat, p))

sns.distplot(train['LotArea']);

Another highly skewed variables **LotArea** has a long left tail. The majority of values are around 10,000 but it maximum value can reach to over 200,000.

In [ ]:
stat, p = shapiro(np.log(train['LotArea']))
print('After log transformation...')
print('Skewness=%.3f' %np.log(train['LotArea']).skew())
print('Statistics=%.3f, p=%.3f' %(stat, p))

sns.distplot(np.log(train['LotArea']));

After we apply the log transformation to LotArea, the skewness is reduced a lot but it is still not a normal distribution.

In [ ]:
stat, p = shapiro(train['MasVnrArea'].dropna())
print('Skewness=%.3f' %train['MasVnrArea'].skew())
print('Statistics=%.3f, p=%.3f' %(stat, p))

sns.distplot(train['MasVnrArea'].dropna());

In [ ]:
masvnrarea_std = (train['MasVnrArea'] - np.mean(train['MasVnrArea'])) / np.std(train['MasVnrArea'])
stat, p = shapiro(masvnrarea_std.dropna())
print('Skewness=%.3f' %masvnrarea_std.skew())
print('Statistics=%.3f, p=%.3f' %(stat, p))

sns.distplot(masvnrarea_std.dropna());

Try the standardization with **MasVnrArea**, but it does not change anything. 

**Now let us take a look at our response variable: SalePrice**

In [ ]:
stat, p = shapiro(train['SalePrice'])
print('Skewness=%.3f' %train['SalePrice'].skew())
print('Statistics=%.3f, p=%.3f' %(stat, p))

sns.distplot(train['SalePrice']);

In [ ]:
stat, p = shapiro(np.log(train['SalePrice']))
print('After log transformation...')
print('Skewness=%.3f' %np.log(train['SalePrice']).skew())
print('Statistics=%.3f, p=%.3f' %(stat, p))

sns.distplot(np.log(train['SalePrice']));

Before log transformation, SalePrice is highly skewed. While log transformation can help with reducing skewness, it can not transform the data to normal distribution. But at least from the plot, we can see that it is improved a lot compared to before.

#### 3. Correlation analysis

In [ ]:
train_num_corr = train[train_num].drop(['Id'], axis=1)
corr = pd.DataFrame(train_num_corr.corr(method = 'pearson')['SalePrice'])
corr.sort_values(['SalePrice'], ascending= False)

OverallQual (Overall material and finish quality), GrLivArea (Above grade (ground) living area square feet), GarageCars (Size of garage in car capacity), GarageArea (Size of garage in square feet), TotalBsmtSF (Total square feet of basement area), 1stFlrSF (First Floor square feet), etc. are highly correlated with our response variable, SalePrice. We can then conclude that area-related variables (including ground living area, garage area, basement area, number of rooms) as well as quality and year (like construction year, remodeled year) are the major continous factors associated with the sale price of a house.

In [ ]:
cmap = sns.cubehelix_palette(light = 0.95, as_cmap = True)
sns.set(font_scale=1.2)
plt.figure(figsize = (11, 11))
sns.heatmap(abs(train_num_corr.corr(method = 'pearson')), vmin = 0, vmax = 1, square = True, cmap = cmap);

From the last row of the heatmap, we can get the same information as from the previous correlation table. Other than this, we can also find the multicollinearity problems with the data, for example, GarageCars and GarageArea, YearBuilt and GarageYrBlt.

### Categorical Variables

#### 1. Check the distribution

For categorical variables, I make a frequency table to show the distribution.

In [ ]:
train_cat = train.select_dtypes(include=['object']).columns
train_cat

In [ ]:
pd.set_option('display.max_rows', 300)
df_output = pd.DataFrame()
for i in range(len(train_cat)):
    c = train_cat[i]
    df = pd.DataFrame({'Variable':[c]*len(train[c].unique()),
                       'Level':train[c].unique(),
                       'Count':train[c].value_counts(dropna = False)})
    df['Percentage'] = 100 * df['Count']  / df['Count'].sum()
    df_output = df_output.append(df, ignore_index = True)
    
df_output

Judged from the frequency table, we can infer that the majority of categorical variables are unbalanced. Several classes even only have 1 or 2 observations. Worsely, Alley, FireplaceQu, PoolQC, Fence, MiscFeature contain missing values over 50%.

#### 2. Correlation with SalePrice

Take several variables for example:

In [ ]:
sns.set(style = 'whitegrid', rc = {'figure.figsize':(10,7), 'axes.labelsize':12})
sns.boxplot(x = 'MSZoning', y = 'SalePrice', palette = 'Set2', data = train, linewidth = 1.5);

**MSZoning**. Most of houses are in the Residential Low Density (79%) and Residential Medium Density (15%) areas. However, there is a large variance within the Residential Low (RL) zone. Floating Village(FV) is a special area where a retirement community was developed and have the highest median price among the all. But there are only 16 observations falling within this category and their prices do not seem quite stable looking at the shape of the box of FV.

In [ ]:
col_order = train.groupby(['Neighborhood'])['SalePrice'].aggregate(np.median).reset_index().sort_values('SalePrice')
p = sns.boxplot(x = 'Neighborhood', y = 'SalePrice', palette = 'Set2', data = train, order=col_order['Neighborhood'], linewidth = 1.5)
plt.setp(p.get_xticklabels(), rotation=45);

**Neighborhood**. Besides the zoning classfication, neighborhood also makes a difference. Houses located at Northridge Heights (NridgHt) have higher sale prices than those in other areas generally but the variance is large. The difference between median price of MeadowV (neighborhood with the lowest house prices) and that of NridgHt is over $200,000.

In [ ]:
sns.boxplot(x = 'HouseStyle', y = 'SalePrice', palette = 'Set2', data = train, linewidth = 1.5);

**HouseStyle**. Locations matter a lot when considering house prices, then what about the characteristics of house itself? Popular house styles are 2 Story (50%) and 1 Story (30%). 2 story and 2.5 story (2nd level finished) houses can be sold at relatively higher prices, around 200,000 dollars, while the prices of 1.5 story (2nd level unfinished) houses are mostly around 110,000 dollars. Notably, for multiple story houses, 2 level finished or unfinished have an obvious relationship with house prices.

In [ ]:
sns.scatterplot(x = 'YearBuilt', y = 'SalePrice', data = train, hue = 'HouseStyle', style = 'HouseStyle', palette = 'colorblind');

After introducing year of built into our plot, we can see that houses built in recent years tend to have higher sale prices, especially since 1960. YearBuilt also has certain association with HouseStyle. We can identify at least three clusters in this plot, blue points (2Story) mostly near the right, yellow (1Story) most in the between, and green (1.5Fin) on the left. 

Intuitively, we assume that the price of house with more stories is higher. For example, when there are 2 story houses built in like 1930, they were sold usually at higher prices than green (1.5Fin) and yellow (1Story) points. But from the box plot we already see that 1.5Fin is less expensive than 1Story. This may be explained by YearBuilt since majority of green 1.5Fin were built between 1900 and 1940 while yellow 1Story between 1940 and 1980 and many were built after 2000.

## Data Preparation

### ** Impute Missing Values **

#### 1. Drop columns where there is a large percentage of missing data

In [ ]:
print(train.isnull().sum())

How to define "a large percentage"? Let us try 15% first.

In [ ]:
# training data
train_missing = pd.DataFrame(train.isnull().sum()/len(train.index) * 100)
train_missing.columns = ['percent']
train_missing.loc[train_missing['percent'] > 15, 'column_select'] = True
train_col_select = train_missing.index[train_missing['column_select'] == True].tolist()
train_col_select

In [ ]:
# test data
test_missing = pd.DataFrame(test.isnull().sum()/len(test.index) * 100)
test_missing.columns = ['percent']
test_missing.loc[test_missing['percent'] > 15, 'column_select'] = True
test_col_select = test_missing.index[test_missing['column_select'] == True].tolist()
test_col_select

We know that Alley, FireplaceQu, PoolQC, Fence, MiscFeature are all categorical variables and their missing values occupy over 50% of total. They should be dropped. But LogFrontage has 1201 rows which I personally decide to keep.

In [ ]:
train_col_select.pop(0)
test_col_select.pop(0)

In [ ]:
train.drop(train_col_select, inplace = True, axis = 1, errors = 'ignore')
test.drop(test_col_select, inplace = True, axis = 1, errors = 'ignore')

train.head()

Note that we orginally have 81 variables for training set and 81 for test set.

In [ ]:
train.shape

In [ ]:
test.head()

In [ ]:
test.shape

#### 2. Use medians for numeric variables and the most frequent values for non-numeric variables to replace NA

In [ ]:
from sklearn.base import TransformerMixin
class MissingDataImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)
        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [ ]:
train_nmissing = MissingDataImputer().fit_transform(train.iloc[:,1:-1])
test_nmissing = MissingDataImputer().fit_transform(test.iloc[:,1:])
train_nmissing.head()

In [ ]:
print(train_nmissing.isnull().sum())

### **Transform the Data**

#### 1. For categorical variables, apply onehot encoding

1)  Check if training data and test data have the same categorical variables

In [ ]:
train_cat = train_nmissing.select_dtypes(include=['object']).columns
test_cat = test_nmissing.select_dtypes(include=['object']).columns
train_cat.difference(test_cat)

2) Create dummy variables

I am not using LabelEncoder because for most categorical variables their values are not in order. So I would prefer one hot encoding. To ensure that the output matrix are in the same order in training and test data, I decide to create the one-hot arrays with get_dummies instead of sklearn OneHotEncoder.

In [ ]:
train_w_dummy = pd.get_dummies(train_nmissing, prefix_sep='_', drop_first=True, columns=train_cat)
test_w_dummy = pd.get_dummies(test_nmissing, prefix_sep='_', drop_first=True, columns=test_cat)

cat_dummies = [col for col in train_w_dummy 
               if '_' in col 
               and col.split('_')[0] in train_cat]

3)  Remove additional variales and add missing variables in test data

In [ ]:
for col in test_w_dummy.columns:
    if ("_" in col) and (col.split("_")[0] in train_cat) and col not in cat_dummies:
        test_w_dummy.drop(col, axis=1, inplace=True)
        
for col in cat_dummies:
    if col not in test_w_dummy.columns:
        test_w_dummy[col] = 0        

4) Make sure that variables in test data have the same order as in training data

In [ ]:
train_cols = list(train_w_dummy.columns[:])
test_w_dummy = test_w_dummy[train_cols]

Remember that we have 76 for training and 75 for test before...

In [ ]:
train_w_dummy.shape

In [ ]:
test_w_dummy.shape

#### 2. For numeric variables, normalize the data

1) Check if training data and test data have the same numeric variables

In [ ]:
train_num = train_nmissing.select_dtypes(exclude=['object']).columns
test_num = test_nmissing.select_dtypes(exclude=['object']).columns
test_num.difference(train_num)

2) Normalize test data: standardization or Yeo-Johnson transformation

As we have experimented before, standardization does not change the shape of the distribution in nature but log transformation do. Log transformation requires data to be positive while Yeo-Johnson transformation supports both positive or negative data. 

I simply choose skewness as the criterion to select between standardization and Yeo-Johnson transformation because I am not training my data on linear regression models. There are of course more rigorous criteria.

In [ ]:
train_num_std = [col for col in train_num if abs(train_w_dummy[col].skew()) <= 1]
train_num_yjt = [col for col in train_num if abs(train_w_dummy[col].skew()) > 1]

In [ ]:
train_num_yjt

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer

# standardization
scaler = StandardScaler().fit(train_w_dummy[train_num_std].values)
train_w_dummy[train_num_std] = scaler.transform(train_w_dummy[train_num_std].values)
test_w_dummy[train_num_std] = scaler.transform(test_w_dummy[train_num_std].values)

# power transform
pt = PowerTransformer().fit(train_w_dummy[train_num_yjt].values)
train_w_dummy[train_num_yjt] = pt.transform(train_w_dummy[train_num_yjt].values)
test_w_dummy[train_num_yjt] = pt.transform(test_w_dummy[train_num_yjt].values)

In [ ]:
test_w_dummy.head()

3. Apply Principal Component Analysis

To address the multicolliearity problem, I apply PCA to decrease the number of variables.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA().fit(train_w_dummy)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components', fontsize = 12)
plt.ylabel('cumulative explained variance', fontsize = 12)
plt.grid(True);

It is clear to notice that the first 100 variables contain nearly 100% of the variance.

In [ ]:
pca = PCA(n_components = 100)
x_train = pca.fit_transform(train_w_dummy) 
x_test = pca.transform(test_w_dummy) 

## **Start Training**

I am not going to use regression models for now because of the assumptions they make on the data. And my transformation is just to speed up convergence during training. Instead, I will try with random forests, boosting, and neural networks.

In [ ]:
# do not forget to log transform our response variable
y_train = np.log(train['SalePrice']).values

y_test_data = pd.read_csv('../input/sample_submission.csv')
y_test = y_test_data['SalePrice'].values

#### 1. Random Forest


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint
rf_base = RandomForestRegressor(n_estimators=400)

# Look at parameters used by our current forest
print('Parameters of base model:')
pprint(rf_base.get_params())

##### Hyperparameter tuning

1) Randomized search

Compared to grid search, randomized search is less time-consuming so we start from a wider range of parameters with randomized search. In this case, 100 sets of parameters are sampled from all combinations to train.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

max_depth = [int(x) for x in np.linspace(40, 100, num = 4)]
max_depth.append(None)
random_params = {'n_estimators': [int(x) for x in np.linspace(start = 400, stop = 1000, num = 4)],
                'max_depth': max_depth,
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4]}

rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_params, n_iter = 100, cv = 3, n_jobs = -1, verbose = 2, random_state = 12)

In [ ]:
rf_random.fit(x_train, y_train)
rf_random.best_params_

In [ ]:
rf_random.best_score_

In [ ]:
# base model result
from sklearn import metrics

rf_base.fit(x_train, y_train)
y_pred_rf_base = np.exp(rf_base.predict(x_test))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_rf_base))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_rf_base))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_rf_base)))

In [ ]:
# random search with best performance parameters
rf_random_best = rf_random.best_estimator_
y_pred_rf_random = np.exp(rf_random_best.predict(x_test))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_rf_random))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_rf_random))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_rf_random)))

Through randomized search, we can see that the performance of the random forest is improved.

2) Grid search

More refined tuning through grid search.

In [ ]:
from sklearn.model_selection import GridSearchCV
grid_params = {'max_depth': [40, 50, 60],
               'min_samples_leaf': [3, 4, 5],
               'min_samples_split': [4, 5, 6],
               'n_estimators': [600, 800]}

rf_grid = GridSearchCV(estimator = rf, param_grid = grid_params, cv = 3, n_jobs = -1, verbose = 2)
rf_grid.fit(x_train, y_train)
rf_grid.best_estimator_

In [ ]:
# grid search with best performance parameters
rf_grid_best = rf_grid.best_estimator_
y_pred_rf_grd = np.exp(rf_grid_best.predict(x_test))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_rf_grd))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_rf_grd))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_rf_grd)))

In [ ]:
#prediction = pd.DataFrame(y_pred_rf_grd, columns = ['SalePrice'])
#result = pd.concat([y_test_data['Id'], prediction], axis = 1)
#result.to_csv('./Predictions.csv', index=False)

Also improved a little bit compared to the model specified using randomized search.

#### 2. XGBoost

In [ ]:
from xgboost import XGBRegressor

xgb_base = XGBRegressor()
xgb_base.fit(x_train, y_train)
y_pred_xgb_base = np.exp(xgb_base.predict(x_test))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_xgb_base))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_xgb_base))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_xgb_base)))

Waiting for parameter optimization...

In [ ]:
#random_params = {}

#xgb_random = RandomizedSearchCV(estimator = xgb_base, param_distributions = random_params, n_iter = 100, cv = 3, n_jobs = -1, verbose = 2, random_state = 12)
#xgb_random.fit(x_train, y_train)
#xgb_random.best_params_

In [ ]:
#xgb_random_best = xgb_random.best_estimator_
#y_pred_xgb_random = np.exp(xgb_random_best.predict(x_test))
#print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_xgb_random))  
#print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_xgb_random))  
#print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_xgb_random)))

#### 3. Neural Networks

Waiting for update...